<a href="https://colab.research.google.com/github/sin-of-sloth/mushroom/blob/master/Mushroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>